# BUILD LLM Bootcamp Day 2

Prerequisite: complete environment setup and package insallations from `llm-day1-notebook.ipynb`

Docker creation instructions also in `llm-day1-notebook.ipynb`

additionally we need into install peft

In [9]:
#!pip install peft

### Import Libraries

In [5]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("snowflake.ml")
logger.setLevel(logging.ERROR)

### Establish Secure Connection

*NOTE: Update [connection.json](connection.json) and set your password, Hugging Face token, and replace '####' with your user number.*

In [6]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DASH
Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 7.40.0
Snowpark for Python version : 1.9.0


### Fine-tune Llama 2 in Snowpark Container Services

There are several steps to starting `FINETUNE_LLM_SERVICE`.

1. Create Compute Pool
2. Create Stage and upload .yaml
3. Create Stage to mount in container
4. Start Service
5. Check Status
6. Connect to Jupyter Endpoint
7. Fine tune model

#### 1. Create Compute Pool
We will use gpu_3

In [ ]:
# create compute pool if it does not already exist. 
session.sql(f"""CREATE COMPUTE POOL {connection_parameters['compute_pool']}
  MIN_NODES = 1
  MAX_NODES = 1
  INSTANCE_FAMILY = GPU_3;""").collect()

#### 2. Create Stage and upload .yaml

In [ ]:
# create YAML_STAGE if it does not already exist and put yaml file in it
session.sql("""create stage YAML_STAGE;""").collect()
session.sql("""PUT file://llm-bootcamp.yaml @YAML_STAGE AUTO_COMPRESS=FALSE OVERWRITE=TRUE;""").collect()

#### 3. Create Stage to mount in container
This stage needs to be encrypted with SNOWFLAKE_SSE. We will also upload our finetuning notebook and a .py file from Meta to faciliate fine tuning. 

In [ ]:
# create llm_workspace if it does not already exist. 
session.sql("""create stage llm_workspace encryption = (type = 'SNOWFLAKE_SSE');""").collect()

In [34]:
# upload finetuning notebook and utils.py file
session.sql("""PUT file://spcs_finetuning/day2-spcs-fine-tunning.ipynb @llm_workspace AUTO_COMPRESS=FALSE;""").collect()
session.sql("""PUT file://spcs_finetuning/ft_datasets/utils.py @llm_workspace/ft_datasets AUTO_COMPRESS=FALSE;""").collect()

#### 4. Start FINETUNE_LLM_SERVICE

In [ ]:
output = session.sql(f"""CREATE SERVICE FINETUNE_LLM_SERVICE
  IN COMPUTE POOL {connection_parameters['compute_pool']}
  FROM @YAML_STAGE
  SPEC='llm-bootcamp';""").collect()

#### 5. Check Status

In [14]:
output2 = session.sql("""SHOW SERVICES;""").collect()
pd.DataFrame(output2)

,name,database_name,schema_name,owner,compute_pool,dns_name,public_endpoints,min_instances,max_instances,auto_resume,created_on,updated_on,comment
0,FINETUNE_LLM_SERVICE,DASH_DB,DASH_SCHEMA,DASH_SPCS,DASH_BUILD_GPU3,finetune-llm-service.dash-schema.dash-db.snowflakecomputing.internal,"{""llm-bootcamp"":""bab5b3e-sfsenorthamerica-build-spcs.snowflakecomputing.app""}",1,1,true,2023-11-15 12:28:38.120000-08:00,2023-11-15 12:28:38.871000-08:00,None
1,SERVICE_2F0D401E835611EE8F5ACE7DB0E7935F,DASH_DB,DASH_SCHEMA,DASH_SPCS,DASH_BUILD_GPU3,service-2f0d401e835611ee8f5ace7db0e7935f.dash-schema.dash-db.snowflakecomputing.internal,{},1,1,true,2023-11-14 17:27:55.588000-08:00,2023-11-14 17:27:56.277000-08:00,None
2,SERVICE_A9B776CE834511EE9C921204C7B8F46D,DASH_DB,DASH_SCHEMA,DASH_SPCS,DASH_BUILD_GPU3,service-a9b776ce834511ee9c921204c7b8f46d.dash-schema.dash-db.snowflakecomputing.internal,{},1,1,true,2023-11-14 15:43:39.294000-08:00,2023-11-14 15:43:39.926000-08:00,None
3,SERVICE_D26762D883E911EE8F5ACE7DB0E7935F,DASH_DB,DASH_SCHEMA,DASH_SPCS,DASH_BUILD_GPU3,service-d26762d883e911ee8f5ace7db0e7935f.dash-schema.dash-db.snowflakecomputing.internal,{},1,1,true,2023-11-15 11:04:52.385000-08:00,2023-11-15 11:04:53.742000-08:00,None
4,SERVICE_DF359FBC833E11EEB4DF42E086352061,DASH_DB,DASH_SCHEMA,DASH_SPCS,DASH_BUILD_GPU3,service-df359fbc833e11eeb4df42e086352061.dash-schema.dash-db.snowflakecomputing.internal,{},1,1,true,2023-11-14 14:41:04.235000-08:00,2023-11-14 14:41:04.875000-08:00,None


In [11]:
eval(session.sql("""CALL SYSTEM$GET_SERVICE_STATUS('DASH_DB.DASH_SCHEMA.FINETUNE_LLM_SERVICE', 100); """).collect()[0][0])

[{'status': 'READY',
  'message': 'Running',
  'containerName': 'llm-bootcamp',
  'instanceId': '0',
  'serviceName': 'FINETUNE_LLM_SERVICE',
  'image': 'sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/image_repo/llm-bootcamp',
  'restartCount': 0,
  'startTime': '2023-11-15T20:28:39Z'}]

#### 6. Connect to jupyter lab endpoint

Use the url below to connect to the jupyter lab endpoint and log in. The remainder of finetunning will be ran from within the jupyter lab endpoint.

You may need to wait a few minutes for the endpoint to be provisioned after status is "READY"

In [13]:
# if this errors, try waiting a few minutes
for k, v in eval(session.sql("""DESCRIBE SERVICE FINETUNE_LLM_SERVICE;""").collect()[0]['public_endpoints']).items():
    if k=='jupyter':
        print(f"{k} url:/ \t{v}/lab")
    else:
        print(f"{k} url: \t{v}")

llm-bootcamp url: 	bab5b3e-sfsenorthamerica-build-spcs.snowflakecomputing.app


#### 7. Fine-tune Llama 2 Model

Once connected to the endpoint, navigate into llm-workspace and open the notebook `day2-fine-tunning.ipynb` and follow the notebook through completion, then return to this notebook.

### Log and Deploy Fine-tuned Llama 2

- Logging and deploying fine-tuned model on this setup (compute and other resources) will take about ~15mins

#### Get weights from Stage

First we need to retreive our fine-tuned weights from our stage `@llm_workspace`

In [33]:
!mkdir output_weights_dir

In [26]:
weight_files = pd.DataFrame(session.sql("LIST @llm_workspace/output_weights_dir").collect())
weight_files

,name,size,md5,last_modified
0,llm_workspace/output_weights_dir/README.md,5447,166b19604020ae26170b43221aba7dfb,"Thu, 16 Nov 2023 03:12:58 GMT"
1,llm_workspace/output_weights_dir/adapter_config.json,496,76a16ee0269ce0b4fb308ae98313dd64,"Thu, 16 Nov 2023 03:12:58 GMT"
2,llm_workspace/output_weights_dir/adapter_model.bin,16822989,db2b8af733dbf68618aaf72baa4f164e-2,"Thu, 16 Nov 2023 03:12:58 GMT"


In [27]:
for file in weight_files.iterrows():
    session.sql(f"GET @{file[1]['name']} file://output_weights_dir").collect()

In [28]:
! cd output_weights_dir && ls -l

total 32888
-rw-r--r--  1 jprusa  staff      5447 Nov 15 20:14 README.md
-rw-r--r--  1 jprusa  staff       496 Nov 15 20:14 adapter_config.json
-rw-r--r--  1 jprusa  staff  16822989 Nov 15 20:14 adapter_model.bin


#### Registery

In [ ]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "FineTunedV1.1"
DEPLOYMENT_NAME = "FINETUNED_LLAMA2"
MODEL_REGISTRY_DB = connection_parameters['database']
MODEL_REGISTRY_SCHEMA = connection_parameters['schema']
COMPUTE_POOL = connection_parameters['compute_pool']

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

##### Delete any existing deployments and models from in Snowpark registry to avoid resource contention during this bootcamp

In [44]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

,MODEL_NAME,MODEL_VERSION,DEPLOYMENT_NAME,CREATION_TIME,TARGET_METHOD,TARGET_PLATFORM,SIGNATURE,OPTIONS,STAGE_PATH,ROLE


In [45]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [46]:
# registry.delete_model(model_name=MODEL_NAME,model_version=MODEL_VERSION,delete_artifact=True)

#### Reference Llama 2 from Fine Tuning

In [10]:
options = llm.LLMOptions(
    token=connection_parameters['huggingface_token'],
    max_batch_size=100,
)
llama_model = llm.LLM(
    model_id_or_path="output_weights_dir",
    options=options
)

In [14]:
%%time
llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

/Users/jprusa/miniconda3/snowpark-ml/lib/python3.10/site-packages/snowflake/ml/_internal/env_utils.py:217: UserWarning: Package requirement torch==2.0.1 specified, while version 2.1.1 is installed. Local version will be ignored to conform to package requirement.
  warnings.warn(
/Users/jprusa/miniconda3/snowpark-ml/lib/python3.10/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:353: UserWarning: Found dependencies specified as pip requirements. This may prevent model deploying to Snowflake Warehouse.
  warnings.warn(


{'name': 'DASH_DB.DASH_SCHEMA.FINETUNED_LLAMA2',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'DASH_BUILD_GPU3', 'num_gpus': 1},
 'details': {'image_name': 'sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/snowml_repo/a043c1a4da636b93ba922effd203c9cfeb449785:latest',
  'service_spec': "spec:\n  container:\n  - env:\n      MODEL_ZIP_STAGE_PATH: /DASH_DB.DASH_SCHEMA.SNOWML_MODEL_B0212BDA842711EE813446C0D41E10B9/model.zip\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n      TARGET_METHOD: infer\n      _CONCURRENT_REQUESTS_MAX: 1\n    image: sfsen

#### Load Datasets

In [19]:
df = pd.read_json("data/transcripts.json", lines=True)
print(f"Total records: {df.shape[0]}")
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

Total records: 1368
Train        : 100
Eval         : 100
Test         : 100


#### Prepare Eval Dataset with Specific Instructions

In [20]:
begin_prompt = \
"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys.
'location': Location string of the caller.
'toy_list': List of toy names from the caller.
### Input:

"""
end_prompt = " [/INST]"

eval_df['input'] = begin_prompt + eval_df['input'] + end_prompt
eval_df = eval_df.reset_index(drop=True)
eval_df.head()

/var/folders/zj/zysg6xyd0yzbpy9kkl0fr4br0000gn/T/ipykernel_21477/1994325739.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df['input'] = begin_prompt + eval_df['input'] + end_prompt


,instruction,input,output,language
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys.\n'location': Location string of the caller.\n'toy_list': List of toy names from the caller.\n### Input:\n\n""caller: Hey, I'm Jack, and I'm looking for a toy for my niece.\nfrosty: Hello Jack! I'm Frosty, and I'd be happy to help you find the perfect toy. Where are you calling from?\ncaller: I'm calling from Cape Town, South Africa.\nfrosty: Thank you, Jack from Cape Town! Can you tell me a bit about your niece's interests so we can find a great toy for her?\ncaller: She's really into animals and likes playing pretend with her toys.\nfrosty: How about the Fisher-Price Little People Mickey and Friends Playset? This playset features animal characters and is perfect for encouraging imaginative play.\ncaller: That sounds nice, but I think she'd prefer something focused on real animals.\nfrosty: In that case, the Calico Critters Pony's Stylish Hair Salon may be just the toy you're looking for. It comes with pony figurines and a cute hair salon for them.\ncaller: Oh, that sounds perfect! Let's go with the pony's hair salon.\nfrosty: Great choice! We've added the Calico Critters Pony's Stylish Hair Salon to your holiday wish list. Before we finish though, can you tell me how you plan to celebrate the holiday season?\ncaller: Sure, I'll be spending the holidays at the beach with my family and enjoying the warm weather.\nfrosty: That sounds amazing! To confirm, your toy list includes the Calico Critters Pony's Stylish Hair Salon. Is that correct?\ncaller: Yes, that's correct. Thank you!+\nfrosty: You're welcome, Jack! It's been a pleasure assisting you. Have a fantastic holiday season!"" [/INST]","{""toy_list"": [""Calico Critters Pony's Stylish Hair Salon""], ""location"": ""Cape Town""}",EN
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys.\n'location': Location string of the caller.\n'toy_list': List of toy names from the caller.\n### Input:\n\n""caller: Hi Frosty, I'm Emily.\nfrosty: Hello Emily! I'm here to help you with your holiday wish list. Where are you calling from?\ncaller: I'm calling from Sydney, Australia.\nfrosty: Thank you, Emily from Sydney! What's the first item you'd like to add to your wish list?\ncaller: I'm looking for a toy that's educational and fun for my son. He's really into science and technology.\nfrosty: How about the Thames & Kosmos Candy Vending Machine STEM experiment kit? It combines science and technology with creating a candy vending machine!\ncaller: That sounds great, but I'd like to explore some more options before deciding.\nfrosty: Sure thing! We also have the LeapFrog Magic Adventures Microscope, which allows kids to explore the world of science up-close with a real working microscope.\ncaller: I think the magic microscope is the one! Let's add that to my wish list.\nfrosty: Fantastic choice! I've added the LeapFrog Magic Adventures Microscope to your list. Before we wrap up, tell me a bit about your favorite way to spend time during the holiday season.\ncaller: I enjoy baking holiday treats and watching seasonal movies with my family.\nfrosty: That sounds delightful! To confirm, your toy list includes the LeapFrog Magic Adventures Microscope. Is everything co

#### Inference on Eval Dataset using fine-tuned Llama 2

In [21]:
llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df).head()

,generated_text
0,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Cape Town, South Africa"",\n""toy_list"": [\n""Calico Critters Pony's Stylish Hair Salon""\n]\n}"
1,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Sydney, Australia"",\n""toy_list"": [\n""LeapFrog Magic Adventures Microscope""\n]\n}"
2,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Auckland, New Zealand"",\n""toy_list"": [\n""Gabbys Dollhouse Cruise Ship""\n]\n}"
3,"Sure, here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Los Angeles, United States"",\n""toy_list"": [\n""Pok\u00e9mon 8-Inch Plush First Partner Three-Pack""\n]\n}"
4,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Dublin, Ireland"",\n""toy_list"": [\n""Playfoam Naturals Shape & Learn Letters & Numbers set""\n]\n}"


#### Clean Up Resources

Delete deployment and the model

In [22]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)
llama_model_ref.delete_model(delete_artifact=True)